# Seed-to-alignment in steps

<a href="https://githubtocolab.com/harmslab/topiary-examples/blob/main/notebooks/seed-to-alignment_api-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook breaks the seed-to-alignment pipeline into individual steps. The goals for this notebook are:
1. Help users understand what topiary is doing in this pipeline.d
2. Provide a starting point for users to customize specific aspects of the pipeline.
3. Demonstrate the topiary API so users can develop their own pipelines. 

The seed-to-alignment pipeline takes a seed dataframe, BLASTs to find homologs, identifies orthologs by reciprocal BLAST, performs sequence quality control, lowers sequence redundancy, and generates a final alignment. This notebook recapitulates the steps run with a single function call in the *01_seed_to_alignment.ipynb* XX notebook or `topiary-seed-to-alignment` script.


## API introduction
The basic idea of the topiary API is to have each function take a topiary
dataframe as an argument and to then return a modified *copy* of that dataframe
as an output. This allows one to write pipelines, as well as easily save out
intermediate steps. The basic flow goes something like:

```
df = topiary.do_something(df,args)
topiary.write_dataframe(df,"current-state.csv")
```
The main topiary functions take a topiary dataframe as their first argument,
other arguments needed by the function, and then return an appropriately
modified copy of the dataframe. Topiary functions generally modify dataframes by
adding columns with new information and/or by setting the `keep` column to
`True` or `False`. The modified dataframe can then be written out to
a csv file to preserve the current state of the dataframe.



## Set up the environment

Run the next two cells to initialize the environment to run topiary. 

In [ ]:
### THIS CELL SETS UP TOPIARY IN A GOOGLE COLAB ENVIRONMENT. 
### IF RUNNING THIS NOTEBOOK LOCALLY, IT MAY BE SAFELY DELETED.

#@title Install software

#@markdown #### Installation requires two steps.

#@markdown 1. Install the software by pressing the _Play_ button on the left.
#@markdown Please be patient. This will take several minutes. <font color='teal'>
#@markdown After the  installation is complete, the kernel will reboot 
#@markdown and Colab will complain that the session crashed. This is normal.</font>
#@markdown <br/>
#@markdown 2. After this cell runs, run the "Initialize environment" cell that follows.

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e: 
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

if RUNNING_IN_COLAB:

    import os
    os.chdir("/content/")

    import urllib.request
    urllib.request.urlretrieve("https://raw.githubusercontent.com/harmslab/topiary-examples/main/notebooks/colab_installer.py",
                              "colab_installer.py")

    import colab_installer
    colab_installer.install_topiary(install_raxml=False,
                                    install_generax=False)

In [ ]:
### IF YOU ARE RUNNING LOCALLY, make sure you installed topiary and
### make sure you activated the topiary conda environment. (If you
### did not start this notebook within that environment, close the
### session, activate the topiary environment, and restart). 

import topiary
import numpy as np
import pandas as pd 

### EVERYTHING AFTER THIS LINE IS IS USED TO SET UP TOPIARY IN A GOOGLE
### COLAB ENVIRONMENT. IF RUNNING THIS NOTEBOOK LOCALLY, THE LINES BELOW
### IN THIS CELL MAY BE SAFELY DELETED. 

#@title Initialize environment

#@markdown  Run this cell to initialize the environment after installation.
#@markdown (This cell can also be run if the kernel dies during a calculation,
#@markdown allowing you to reload modules without having to
#@markdown reinstall.) Re-run this cell if you have to re-run any subsequent
#@markdown cells so that your calculations are in the correct directory.

#@markdown We recommend setting up a working directory on your google drive. This is a 
#@markdown convenient way to pass files to topiary and will allow you to save
#@markdown your work. For example, if you type `topiary_work` into the form
#@markdown field below, topiary will save all of its calculations in the 
#@markdown `topiary_work` directory in MyDrive (i.e. the top directory at
#@markdown https://drive.google.com). This script will create the directory if 
#@markdown it does not already exist. If the directory already exists, any files
#@markdown that are already in that directory will be available to topiary. You could, 
#@markdown for example, put a file called `seed.csv` in `topiary_work` and then
#@markdown access it as "seed.csv" in all cells below.
#@markdown <br/><br/>
#@markdown Note: Google may prompt you for permission to access the drive. 
#@markdown To work in a temporary colab environment, leave this blank. 

# Select a working directory on google drive
google_drive_directory = "" #@param {type:"string"}

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e: 
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

if RUNNING_IN_COLAB:

    import os
    os.chdir("/content/")

    topiary._in_notebook = "colab"
    import colab_installer
    colab_installer.initialize_environment()
    colab_installer.mount_google_drive(google_drive_directory)

## Construct a seed dataset
The first step in a topiary ASR calculation is to construct a seed dataset. This dataset defines protein family members of interest and the distribution of these proteins across species. Topiary uses this seed dataset to automatically find and download sequences to put into the alignment and, ultimately, evolutionary tree. An example for the LY86/LY96 protein family, a pair of closely related innate immune proteins, is shown below. 

name | species      | sequence   | aliases
---- | ------------ | ---------- | -------------------------------------------------------------------------------------------
LY96 | Homo sapiens | MLPFLFF... | ESOP1;Myeloid Differentiation Protein-2;MD-2;lymphocyte antigen 96;LY-96
LY96 | Danio rerio  | MALWCPS... | ESOP1;Myeloid Differentiation Protein-2;MD-2;lymphocyte antigen 96;LY-96
LY86 | Homo sapiens | MKGFTAT... | Lymphocyte Antigen 86;LY86;Myeloid Differentiation Protein-1;MD-1;RP105-associated 3;MMD-1
LY86 | Danio rerio  | MKTYFNM... | Lymphocyte Antigen 86;LY86;Myeloid Differentiation Protein-1;MD-1;RP105-associated 3;MMD-1

[Download the full spreadsheet](https://raw.githubusercontent.com/harmslab/topiary-examples/main/data/ly86-ly96.csv)

For a description of how to prepare this table, see the [topiary documentation](https://topiary-asr.readthedocs.io/en/latest/protocol.html).

In [ ]:
### IF RUNNING LOCALLY: set `seed_spreadsheet_file =` to point to your desired csv or xlsx file. 
### Alternatively, you can set a `seed_df` to point to a pandas dataframe holding the
### seed dataset. 

seed_spreadsheet_file = "https://raw.githubusercontent.com/harmslab/topiary-examples/main/data/ly86-ly96.csv"
seed_df = None

# -----------------------------------------------------------------------------
# COLAB SPECIFIC BLOCK

#@title Load seed dataset

#@markdown Before running this cell, specify either: 
#@markdown + A file containing a seed dataset in your working
#@markdown directory (your google drive specified above).
#@markdown The default input file is an example LY86/LY96 seed dataset.
#@markdown + Select `upload_file` to upload a file directly from your computer. 

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e: 
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

if RUNNING_IN_COLAB:

    seed_spreadsheet_file = "https://raw.githubusercontent.com/harmslab/topiary-examples/main/data/ly86-ly96.csv" #@param {type:"string"}
    upload_file = False #@param {type:"boolean"}

    if issubclass(type(seed_spreadsheet_file),str):
        seed_spreadsheet_file = seed_spreadsheet_file.strip()

    if seed_spreadsheet_file != "" and upload_file:
        err = "Please give a seed_spreadsheet_file OR select upload file\n"
        raise ValueError(err)

    if seed_spreadsheet_file == "" and not upload_file:
        err = "Please either give a seed_spreadsheet_file or select upload file\n"
        raise ValueError(err)

    if upload_file:

        try:
            from google.colab import files
            uploaded_files = files.upload()
            keys = list(uploaded_files.keys())
            seed_spreadsheet_file = keys[0] #uploaded_files[keys[0]]
        except ImportError:
            pass

# END COLAB SPECIFIC BLOCK
# -----------------------------------------------------------------------------

# Read seed_df from the input file
if seed_df is None:

    try:
        seed_df = pd.read_csv(seed_spreadsheet_file)
    except:
        try:
            seed_df = pd.read_excel(seed_spreadsheet_file)
        except:
            err = f"Could not read {seed_spreadsheet_file}. This should be a csv or xlsx file\n"
            raise ValueError(err)

seed_df 

## Construct an initial dataset using BLAST

At this point, we have a seed dataset as a pandas DataFrame (`seed_df`; see output of last cell). For the next step, we BLAST against user-specified database(s) to find homologs of the proteins in our seed dataset. 

### Inputs:
+ `seed_df`: The only required input to the following function is the seed dataframe.
+ Other arguments to this function give the user control over how BLASTing is done. For information about these parameters, see the [topiary documentation](https://topiary-asr.readthedocs.io/en/latest/topiary.io.html#topiary.io.seed.df_from_seed).

### Outputs:
+ `df`: A new topiary dataframe with the BLAST hits given by the sequences in `seed_df`. 
+ `key_species`: A list of the key species in the seed dataframe.
+ `paralog_patterns`: A dictionary of [regular expressions](https://docs.python.org/3/library/re.html) that allow topiary to look for reciprocal BLAST hits that match the paralog aliases from the see dataframe. See documentation on [paralog patterns](https://topiary-asr.readthedocs.io/en/latest/data_structures.html#paralog-patterns) for details on this dictionary.
+ `species_aware`: Whether or not to do the remaining analysis in a species-aware fashion. By default, the analysis will be species aware for non-microbial datasets and not be species aware for microbial datasets. 




In [ ]:
# These function arguments are the default values. The function should run with these
# values. For more information about the parameters, see:
# https://topiary-asr.readthedocs.io/en/latest/topiary.io.html#topiary.io.seed.df_from_seed

df, key_species, paralog_patterns, species_aware = topiary.df_from_seed(seed_df=seed_df,
                                                                        ncbi_blast_db="nr",
                                                                        local_blast_db=None,
                                                                        blast_xml=None,               
                                                                        move_mrca_up_by=2,
                                                                        species_aware=None,
                                                                        hitlist_size=5000,            
                                                                        e_value_cutoff=0.001,         
                                                                        gapcosts=(11,1),
                                                                        num_ncbi_blast_threads=1,       
                                                                        num_local_blast_threads=-1,
                                                                        keep_blast_xml=False)  
    
# Save out the dataframe so we can keep track of our changes. 
topiary.write_dataframe(df,"01_initial-dataframe.csv")
df

### Things to look for in your output dataframe:

1. The dataframe has your seed sequences plus the BLAST results. (There should be *a lot* more sequences than your seed dataframe).
2. If you scroll to the right, there is a ton of information about each sequence (accession, sequence, e_value, etc.). The columns of the dataframe can be accessed using the command `df.columns`.
3. The `keep` column is special. Topiary does not delete sequences from the analysis, but instead sets `keep = False` for sequences that do not pass quality control. All sequences should have `keep = True` at the moment because we have not done quality control yet.
4. The sequences from your seed dataframe are at the top of this output dataframe. They have two columns set to `True`: `always_keep` (meaning they will not be deleted) and `key_species` (meaning they are from a key species). **PRO TIP**: If you want to make sure a sequence is not deleted, you can set `always_keep = True` for that sequence and topiary will not delete it, regardless of downstream quality control steps. 
5. The `ott` and `resolvable` columns indicate the species identifier on the Open Tree of Life database (`ott`) and whether the species can be unambiguously placed on a species tree (`resolvable`). For a species aware reconstruction, only sequences with `resolvable = True` can be included in the main phylogenetic analysis. 

## Identify each hit by reciprocal BLAST

Once the initial dataset is constructed, topiary identifies each hit by reciprocal BLAST. The cell below downloads the proteomes for the key species from the NCBI, constructs a local BLAST databse, and then uses each BLAST hit from above as a query against this BLAST database from the key species. It then identifies hits whose descriptions match the text patterns from `paralog_pattern` generated above. Any sequence that does not match at least one pattern has `keep` set to `False` as the sequence is likely a paralog rather than ortholog to at least one of the sequences in the seed dataset. 

### Inputs:
+ This cell uses three inputs from the previous cell:
    + `df`: The topiary dataframe holding the BLAST hits. 
    + `key_species`: The key species in your dataset.
    + `paralog_patterns`: A dictionary of regular expressions used to search the key proteomes from reciprocal BLAST hits.
+ For detailed descriptions of the arguments to the three functions used, see the documentation:
    + [topiary.ncbi.get_proteome](https://topiary-asr.readthedocs.io/en/latest/topiary.ncbi.entrez.html#topiary.ncbi.entrez.proteome.get_proteome)
    + [topiary.ncbi.make_blast_db](https://topiary-asr.readthedocs.io/en/latest/topiary.ncbi.blast.html#topiary.ncbi.blast.make.make_blast_db)
    + [topiary.recip_blast](https://topiary-asr.readthedocs.io/en/latest/topiary.ncbi.blast.html#module-topiary.ncbi.blast.recip)


### Output:
+ An updated topiary dataframe with reciprocal BLAST results.

In [ ]:
# If you want to restart the pipeline *without* running the pipeline steps above, 
# please uncomment the following lines. If you want to force this step to be 
# species aware (or not) set species_aware=True (or False). 

# _, key_species, paralog_patterns, species_aware = topiary.io.read_seed(seed_spreadsheet_file,
#                                                                        species_aware=None)
# df = topiary.read_dataframe("01_initial-dataframe.csv")


# Create a list of proteomes to download
key_proteomes = []
for k in key_species:
    get_proteome = topiary.ncbi.get_proteome(species=k)
    key_proteomes.append(get_proteome)

# Construct a blast database from the downloaded proteomes
local_recip_blast_db = "local_recip_blast_db"
topiary.ncbi.make_blast_db(key_proteomes,local_recip_blast_db)

# Do reciprocal BLAST against the proteomes from the key species
df = topiary.recip_blast(df,
                         paralog_patterns=paralog_patterns,
                         local_blast_db=local_recip_blast_db,
                         ncbi_blast_db=None,
                         ncbi_taxid=None,
                         ignorecase=True,
                         min_call_prob=0.95,
                         partition_temp=1,
                         drop_combo_fx=0.9,
                         use_start_end=True,
                         hitlist_size=10,
                         e_value_cutoff=0.01,
                         gapcosts=(11,1),
                         num_threads=-1,
                         keep_blast_xml=False)

topiary.write_dataframe(df,"02_recip-blast-dataframe.csv")
df

### Things to look for in the output dataframe
+ This dataframe should have the same number of sequences as the input dataframe; however, some of the sequences now have `keep = False`. This is because they failed to find reciprocal BLAST hits. 
+ The dataframe has five new columns (`recip_found_paralog`, `recip_hit`, `recip_paralog`, `recip_prob_match`, and `recip_bit_score`). For a full description of what these mean, see the [function documentation](https://topiary-asr.readthedocs.io/en/latest/topiary.ncbi.blast.html#module-topiary.ncbi.blast.recip). The most important two columns are `recip_paralog` (which indicates the paralog call) and `recip_prob_match` (which indicates our confidence in the call, where `1.0` is highly confident). 

## Remove redundant sequences. 

BLAST typically finds many more sequences than are necessary or practical for a standard phylogenetic analysis. We must therefore select sequences that sample the diversity in the dataset without compromising our ability to infer ancestors. Topiary selects a subset of sequences using a combination of taxonomy, sequence identity, and sequence quality (see [documentation](https://topiary-asr.readthedocs.io/en/latest/pipelines.html#seed-to-alignment)). The following code block decides how many sequences to put in the final alignment (`target_seq_number`), removes highly similar sequences from within each sequences (`shrink_in_species`), lowers overall redundancy (`shrink_redundant`), and removes the worst aligning sequences (`shrink_aligners`). 


### Inputs:
+ This cell uses three inputs from the previous cell:
    + `df`: The topiary dataframe holding current dataset
    + `seqs_per_column`: number of sequences to include in the alignment per column in the seed dataset sequences. (If the largest seed sequence was 100 amino acids, a value of 2 would create an alignment with 200 sequences)
+ For detailed descriptions of the arguments to the functions used, see the documentation:
    + [topiary.quality.shrink_in_species](https://topiary-asr.readthedocs.io/en/latest/topiary.quality.html#topiary.quality.shrink.shrink_redundant)
    + [topiary.quality.redundancy.find_redundancy_cutoff](https://topiary-asr.readthedocs.io/en/latest/topiary.quality.html#topiary.quality.redundancy.find_redundancy_cutoff)
    + [topiary.quality.shrink_redundant](https://topiary-asr.readthedocs.io/en/latest/topiary.quality.html#topiary.quality.shrink.shrink_redundant)
    + [topiary.quality.shrink_aligners](https://topiary-asr.readthedocs.io/en/latest/topiary.quality.html#topiary.quality.shrink.shrink_aligners)


### Output:
+ An updated topiary dataframe with lower sequence redundancy



In [ ]:
# If you want to restart the pipeline *without* running the pipeline steps above, 
# please uncomment the following lines. If you want to force this step to be 
# species aware (or not) set species_aware=True (or False). 

# _, _, _, species_aware = topiary.io.read_seed(seed_spreadsheet_file,species_aware=None)
# df = topiary.read_dataframe("02_recip-blast-dataframe.csv")


# Determine how many sequences we are aiming for. This will aim to have 1 
# sequence each site in the key sequence. 
seqs_per_column = 1
key_sequences = df.loc[df.key_species,"sequence"]
target_seq_number = int(np.round(np.max([len(s) for s in key_sequences])*seqs_per_column*1.1,0))

# The first step is to remove highly similar sequences from the same species.
# This removes things like splice variants, super recent duplications, etc.
df = topiary.quality.shrink_in_species(df,
                                       redundancy_cutoff=0.98)
topiary.write_dataframe(df,"03.0_shrunk-in-species.csv")

# If we are not species aware, we need to find a redundnacy cutoff that
# will yield approximately the right number of sequences. 
if not species_aware:
    redundancy_cutoff = topiary.quality.redundancy.find_redundancy_cutoff(df,
                                                                          target_seq_number=target_seq_number)
else:
    redundancy_cutoff = 0.98

# Remove redundant sequences. If species aware, this will remove sequences
# using the sequence budgeting strategy described in Figure 4 of the 
# manuscript. (Redundancy cutoff will be ignored). Otherwise, remove based
# on high sequence identity. 
df = topiary.quality.shrink_redundant(df,
                                      paralog_column="recip_paralog",
                                      species_tree_aware=species_aware,
                                      weighted_paralog_split=False,
                                      merge_block_size=50,
                                      redundancy_cutoff=redundancy_cutoff)
topiary.write_dataframe(df,"03.1_shrunk-redundant.csv")

# Remove the worst-aligning subset of sequences. 
df = topiary.quality.shrink_aligners(df,
                                     target_seq_number,
                                     paralog_column="recip_paralog",
                                     species_tree_aware=True,
                                     weighted_paralog_split=False,
                                     sparse_column_cutoff=0.80,
                                     align_trim=(0.05,0.95))
topiary.write_dataframe(df,"03.2_shrunk-on-alignment.csv")
df

### Things to look for in the output dataframe
The primary thing to note is that many more of the sequences will now have `keep = False`. These sequences were considered redundant and removed. If you want to bring back any sequences at this point, set `keep = True` for the row in question. This dataframe will also have several new columns indicating the relative quality of different sequences. 

## Alignment
Topiary uses [Muscle5](https://www.drive5.com/muscle/) with its default parameters to generate a first draft of the multiple sequence alignment.

### Inputs:
+ This function requires only a topiary dataframe (`df` from the previous cell). 
+ The remaining arguments can bse used to control how muscle does the alignment. For a detailed description of how they work, see [topiary.muscle.align](https://topiary-asr.readthedocs.io/en/latest/topiary.muscle.html#module-topiary.muscle.muscle)

### Output:
+ An updated topiary dataframe with aligned sequences. 

In [ ]:
# If you want to restart the pipeline *without* running the pipeline steps above, 
# please uncomment the following line.
# df = topiary.read_dataframe("03.2_shrunk-on-alignment.csv")

df = topiary.muscle.align(input_seqs=df,
                          output_fasta=None,
                          super5=False,
                          silent=False,
                          muscle_cmd_args=[],
                          muscle_binary="muscle")
topiary.write_dataframe(df,"04_aligned-dataframe.csv")

# If you want to write out the alignment, you can uncomment the following line:
# topiary.io.write_fasta(df,"alignment.fasta") 

df

### Things to look for in the output dataframe

This function will create a new `alignment` column that has the sequences aligned.

## Polish alignment and re-align

In this final step, we remove sequences that have long insertions or are missing large chunks of the sequence. This prevents these poorly aligned sequences from disrupting the alignment of other sequences in the dataset. For details on how poorly aligned sequences are identified, see the [topiary documentation](https://topiary-asr.readthedocs.io/en/latest/topiary.quality.html#module-topiary.quality.polish). After removing poorly aligned sequences, this function aligns the sequences from scratch using Muscle5.  

### Inputs:
+ This function requires only a topiary dataframe (`df` from the previous cell). 
+ The remaining arguments can bse used to control how the alignment polishing is done. For a detailed description of the parameters and how they work, see [topiary.quality.polish](https://topiary-asr.readthedocs.io/en/latest/topiary.quality.html#module-topiary.quality.polish). 

### Output:
+ An updated topiary dataframe poorly aligned sequences set to False. 


In [ ]:
# If you want to restart the pipeline *without* running the pipeline steps above, 
# please uncomment the following line.
# df = topiary.read_dataframe("04_aligned-dataframe.csv")

# Run the polishing step
df = topiary.quality.polish_alignment(df,
                                      realign=True,
                                      sparse_column_cutoff=0.80,
                                      align_trim=(0,1),
                                      fx_sparse_percentile=0.90,
                                      sparse_run_percentile=0.90,
                                      fx_missing_percentile=0.90)
topiary.write_dataframe(df,"05_clean-aligned-dataframe.csv")

df

## Load in edited alignment

In the [topiary protocol](file:///Users/harmsm/work/programming/git-clones/topiary/docs/build/html/protocol.html#visually-inspect-and-possibly-edit-the-alignment) we describe how we inspect and manually edit alignments. To do this, we need to write out a fasta file that can be loaded into a sequence viewer/editor. After doing those edits, we need to load the alignment back into the `alignment` column of our dataframe. The following two cells show how to do this. 

In [ ]:
# If you want to restart the pipeline *without* running the pipeline steps above, 
# please uncomment the following line.
# df = topiary.read_dataframe("05_clean-aligned-dataframe.csv")

# Write a fasta file
topiary.write_fasta(df,
                    out_file="06_alignment.fasta",
                    seq_column="alignment",
                    label_columns=["species","recip_paralog"])

In [ ]:
# If you want to restart the pipeline *without* running the pipeline steps above, 
# please uncomment the following line.
# df = topiary.read_dataframe("05_clean-aligned-dataframe.csv")

# Change this to point to a fasta file you edited and saved somewhere
edited_fasta_file = "06_alignment.fasta" 

df = topiary.read_fasta_into(df,edited_fasta_file)
topiary.write_dataframe(df,"07_final-dataframe.csv")
df

The file *07_final-dataframe.csv* is now ready for an ASR inference. 